# Курсовой проект.

In [1]:
# Подключение библиотек и скриптов

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Пути к директориям и файлам

TRAINT_DATASET_PATH = 'train.csv'
TEST_DATASET_PATH = 'test.csv'
PREPARED_TRAINT_DATASET_PATH = 'train_prepared.csv'
PREPARED_TEST_DATASET_PATH = 'test_prepared.csv'

In [3]:
# Загрузка данных

df = pd.read_csv(TRAINT_DATASET_PATH)
df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [4]:
df.shape

(10000, 20)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [6]:
# Функция для очистки данных

def data_preparation(df_input):
    
    df_output = df_input.copy()
        
    df_output.drop('Id', axis = 'columns', inplace = True)    
        
    Square = dict(df_output.groupby('Rooms')['Square'].median())
    idx = (df_output['Square'] < 15) & (df_output['Rooms'] < 2)
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'Rooms'].apply(lambda x: Square[x])
    
    idx = (df_output['Square'] < 30) & (df_output['Rooms'] > 1)
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'Rooms'].apply(lambda x: Square[x])
    
    idx = (df_output['Square'] < df_output['LifeSquare'])
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'LifeSquare']

    #LifeSquare = dict(df_output.groupby('Rooms')['LifeSquare'].median())
    #idx = (df_output['LifeSquare'].isnull())
    #df_output.loc[idx, 'LifeSquare'] = df_output.loc[idx, 'Rooms'].apply(lambda x: LifeSquare[x])

    df_output.loc[df_output['Rooms'] == 0, 'Rooms'] = 1
    df_output.loc[df_output['Rooms'] > 6, 'Rooms'] = 1
    
    idx = (df_output['Rooms'] < 1)
    df_output.loc[idx, 'KitchenSquare'] = 0.0
    
    KitchenSquare = dict(df_output.groupby('Rooms')['KitchenSquare'].median())
    idx = (df_output['Rooms'] > 0)
    df_output.loc[idx, 'KitchenSquare'] = df_output.loc[idx, 'Rooms'].apply(lambda x: KitchenSquare[x])
    
    idx = (df_output['HouseFloor'] < df_output['Floor'])
    df_output.loc[idx, 'HouseFloor'] = df_output.loc[idx, 'Floor']
    
    idx = (df_output['HouseYear'] >= 2019)
    df_output.loc[idx, 'HouseYear'] = 2019
   
    df_output['Healthcare_1'].fillna(df_output['Healthcare_1'].median(), inplace=True)
    
    #df_output['Ecology_2'] = df_output['Ecology_2'].apply(lambda x: 1 if x == 'A' else 0)
    #df_output['Ecology_3'] = df_output['Ecology_3'].apply(lambda x: 1 if x == 'A' else 0)
    #df_output['Shops_2'] = df_output['Shops_2'].apply(lambda x: 1 if x == 'A' else 0)
    
    # Выгоднее всего оказалось удалить столбцы с этими признаками:
    df_output.drop('Ecology_2', axis = 'columns', inplace = True)
    df_output.drop('Ecology_3', axis = 'columns', inplace = True)
    df_output.drop('Shops_2', axis = 'columns', inplace = True)
    df_output.drop('LifeSquare', axis = 'columns', inplace = True)
    
    return df_output

In [7]:
df_prep = data_preparation(df)

In [8]:
X = df_prep.drop(labels=['Price'], axis=1)
y = df_prep['Price'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [10]:
# проведем настройку гиперпараметров модели с помощью RandomizedSearchCV

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

random_grid = {
    'n_estimators': np.arange(200, 501, 20),
    'max_depth': np.arange(2, 51, 2),
    'max_features': [0.5, 0.6, 0.7, 0.8, 0.9],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=50,
    scoring='r2',
    cv=10,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print(random_search.best_score_)
print(random_search.best_params_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 10.6min finished


0.7338069913046712
{'n_estimators': 360, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.6, 'max_depth': 32}


In [11]:
# измерим качество подобранной модели на отложенной выборке

y_pred_train = random_search.best_estimator_.predict(X_train)
y_pred_test = random_search.best_estimator_.predict(X_test)

score_train = r2_score(y_train, y_pred_train)
score_test = r2_score(y_test, y_pred_test)

print(f'train: {np.round(score_train, 4)}')
print(f'test: {np.round(score_test, 4)}')

train: 0.9207
test: 0.7233


In [12]:
# обучим подобранную модель на всех тренировочных данных

model = RandomForestRegressor(
    n_estimators=360,
    max_depth=32,
    max_features=0.6,
    min_samples_leaf=2,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X, y)

scores = cross_val_score(model, X, y, scoring='r2', cv=10, n_jobs=-1)

print(f'cv mean: {np.round(np.mean(scores), 4)}')
print(f'cv std:  {np.round(np.std(scores), 4)}')

cv mean: 0.74
cv std:  0.0237


In [13]:
# загрузим данные из тестового файла

data_test = pd.read_csv(TEST_DATASET_PATH)

In [14]:
data_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [15]:
data_test_prep = data_preparation(data_test)

In [16]:
y = model.predict(data_test_prep)

In [17]:
# запишем id объекта и предсказанную цену в датафрейм

predictions = pd.DataFrame({
    'Id': data_test['Id'],
    'Price': y
})

In [18]:
predictions.head()

,Id,Price
0,725,160562.033411
1,15856,224635.746037
2,5480,299662.205547
3,15664,350312.780131
4,14275,145383.288572


In [19]:
# сохраним предсказания в файл

predictions.to_csv('MDorozhkin_predictions.csv', sep=',', index=False, encoding='utf-8')

In [20]:
# Score на kaggle.com получился 0.74832